# Curbage Recognition Experiment Set Up
### Zachariah M. Reagh & Angelique I. Delarazan
#### 12/01/2022
Edited: Changed LDI formula from p(New|Lure) - p(New|Target) to p(New|Lure) - p(Old|Foil) to fit reviewer suggestion 

Generates csv files: 
1. Calculates hits, misses, false, alarms, and correct rejects for each subject for each domain type (Narrative v. Perceptual)
2. Output csv files based on group (Younger v. Older Adults)
3. Appends groups and outpust csv files for further analyses

## Set Up

In [2]:
import os
import csv
from itertools import islice
import pandas as pd
import scipy.stats
from scipy.stats import norm
import math
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels
import numpy as np

In [5]:
base_dir = '/Volumes/GAMMA/curbage_behavioral/experiment1/'
data_dir = os.path.join(base_dir, f'data/recognition/')
output_dir = '/Volumes/GAMMA/curbage_recognition/reviews/analyses/data/'

## Calculate hits, misses, false alarms, and correct rejections

### Older Adults

In [6]:
# Read in older adults' individual data and create a list
sublist_older = pd.read_csv(os.path.join(base_dir, 'code/sub-all_group-older_desc-subject_list.txt'), header=None)
sublist_older = sublist_older[0].values.tolist()
sublist_older

[201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221]

### Narrative Domain Task

In [7]:
# Loop through all subjects on list and create output file, calculating subject's performance for the Narrative Task
Z = norm.ppf

sub_all_group_older_desc_narrative = pd.DataFrame(columns=['subject', 'target_hit', 'lure_cr','foil_cr'])

for subject in sublist_older: 
    narrative_summary = open(os.path.join(output_dir, f'sub-{subject}_group-older_task-recognition_desc-narrative_results.txt'), 'w')
    narrative_summary.write('subject: %d\n' %(subject))
    narrative_summary.write('performance_summary\n\n\n\nraw_data\n\n')
    narrative_summary.write('target,target_hit,target_miss,lure,lure_cr,lure_fa,foil,foil_cr,foil_fa,junk\n')
    
    performance=[]
    target=0
    target_hit=0
    target_miss=0
    lure=0
    lure_cr=0
    lure_fa=0
    foil=0
    foil_cr=0
    foil_fa=0
    junk=0
    
    with open(os.path.join(data_dir, f'sub-{subject}_group-older_task-recognition_desc-narrative.txt')) as fd:
        for line in islice(csv.reader(fd), 5, None):
            performance.append([line[0],line[1],line[2],line[3],line[4]])
            if 't' in line[1]:
                target+=1
            if 't' in line[1] and line[3]=='t':
                target_hit+=1
            if 't' in line[1] and line[3]=='f':
                target_miss+=1
            # Start move to lure trials
            if 's' in line[1]:
                lure+=1
            if 's' in line[1] and line[3]=='t':
                lure_fa+=1
            if 's' in line[1] and line[3]=='f':
                lure_cr+=1
            if 'n' in line[1]:
                foil+=1
            if 'n' in line[1] and line[3]=='t':
                foil_fa+=1
            if 'n' in line[1] and line[3]=='f':
                foil_cr+=1
            if line [1] == '':
                junk+=1
            
    target_hit = target_hit
    target_miss = target_miss
    lure_cr = lure_cr
    lure_fa = lure_fa
    target_hit_rate = target_hit / (target)
    fa_cr_rate = foil_cr / (foil)
    target_miss_rate = target_miss/target
    lure_cr_rate = lure_cr/lure
    lure_fa_rate = lure_fa/lure
    foil_cr_rate = foil_cr/foil
    foil_fa_rate = foil_fa/foil
    cr = lure_cr + foil_cr
    cr_rate = (lure_cr + foil_cr)/(lure + foil)
    fa = lure_fa + foil_fa
    fa_rate = (lure_fa + foil_fa)/(lure + foil)
    
    if target_hit_rate == 1: 
        target_hit_rate = (target - 0.5) / (target)
    if target_hit_rate == 0: 
        target_hit_rate = 0.5 / (target)
        
    if foil_fa_rate == 1: 
        foil_fa_rate = (foil - 0.5) / (foil)
    if foil_fa_rate == 0: 
        foil_fa_rate = 0.5 / (foil)    
    
    dprime = Z(target_hit_rate) - Z(foil_fa_rate)
    dprime_lures = Z(target_hit_rate) - Z(lure_fa_rate)
    
    LDI = lure_cr_rate - target_miss_rate
    
    LDI_reviewer = lure_cr_rate - foil_fa_rate
    
    beta = math.exp((Z(fa_rate)**2 - Z(target_hit_rate)**2) / 2)
    c = -(Z(target_hit_rate) + Z(fa_rate)) / 2
    ad = norm.cdf(dprime / math.sqrt(2))
    
    # Append our big data frame with each subject's data
    
    sub_all_group_older_desc_narrative = sub_all_group_older_desc_narrative.append(pd.DataFrame\
    ({'subject': subject, 'target_hit': target_hit, 'target_miss': target_miss,\
      'lure_cr': lure_cr, 'lure_fa': lure_fa,\
      'foil_cr': foil_cr, 'foil_fa':foil_fa,\
      'target_hit_rate': target_hit_rate, 'lure_cr_rate': lure_cr_rate,'foil_cr_rate': foil_cr_rate,\
      'lure_fa_rate': lure_fa_rate, 'foil_fa_rate':foil_fa_rate,\
      'cr': cr, 'cr_rate': cr_rate, 'fa': fa,'fa_rate': fa_rate,\
      'dprime':dprime, 'dprime_lures':dprime_lures, 'LDI':LDI, 'LDI_reviewer':LDI_reviewer, 'beta':beta, 'c':c, 'ad':ad}, index=[0]), ignore_index=True)

    #Append performance summary file
    narrative_summary.write("%d,%d,%d,%d,%d,%d,%d,%d,%d,%d\n\n\n\n" %(target,target_hit,target_miss,lure,lure_cr,lure_fa,foil,foil_cr,foil_fa,junk))
    narrative_summary.write("targets\n\ntarget_hit_rate,target_miss_rate\n")
    narrative_summary.write("%.3f,%.3f\n\n\n"%(target_hit_rate,target_miss_rate))
    narrative_summary.write("lures\n\nlure_cr_rate,lure_fa_rate\n")
    narrative_summary.write("%.3f,%.3f\n\n\n"%(lure_cr_rate,lure_fa_rate))
    narrative_summary.write("foils\n\nfoil_cr_rate,foil_fa_rate\n")
    narrative_summary.write("%.3f,%.3f\n\n\n"%(foil_cr_rate,foil_fa_rate))
    narrative_summary.close()
    print("narrative_summary created successfully for %s" %(subject))

narrative_summary created successfully for 201
narrative_summary created successfully for 202
narrative_summary created successfully for 203
narrative_summary created successfully for 204
narrative_summary created successfully for 205
narrative_summary created successfully for 206
narrative_summary created successfully for 207
narrative_summary created successfully for 208
narrative_summary created successfully for 209
narrative_summary created successfully for 210
narrative_summary created successfully for 211
narrative_summary created successfully for 212
narrative_summary created successfully for 213
narrative_summary created successfully for 214
narrative_summary created successfully for 215
narrative_summary created successfully for 216
narrative_summary created successfully for 217
narrative_summary created successfully for 218
narrative_summary created successfully for 219
narrative_summary created successfully for 220
narrative_summary created successfully for 221


### Perceptual Domain Task

In [8]:
sub_all_group_older_desc_perceptual = pd.DataFrame(columns=['subject', 'target_hit', 'lure_cr','foil_cr'])

for subject in sublist_older: 
    perceptual_summary = open(os.path.join(output_dir, f'sub-{subject}_group-older_task-recognition_desc-perceptual_results.txt'), 'w')
    perceptual_summary.write('subject: %d\n' %(subject))
    perceptual_summary.write('performance_summary\n\n\n\nraw_data\n\n')
    perceptual_summary.write('target,target_hit,target_miss,lure,lure_cr,lure_fa,foil,foil_cr,foil_fa,junk\n')
    
    performance=[]
    target=0
    target_hit=0
    target_miss=0
    lure=0
    lure_cr=0
    lure_fa=0
    foil=0
    foil_cr=0
    foil_fa=0
    junk=0
    
    with open(os.path.join(data_dir, f'sub-{subject}_group-older_task-recognition_desc-perceptual.txt')) as fd:
        for line in islice(csv.reader(fd), 5, None):
            performance.append([line[0],line[1],line[2],line[3],line[4]])
            if 't' in line[1]:
                target+=1
            if 't' in line[1] and line[3]=='t':
                target_hit+=1
            if 't' in line[1] and line[3]=='f':
                target_miss+=1
            # Start move to lure trials
            if 's' in line[1]:
                lure+=1
            if 's' in line[1] and line[3]=='t':
                lure_fa+=1
            if 's' in line[1] and line[3]=='f':
                lure_cr+=1
            if 'd' in line[1]:
                foil+=1
            if 'd' in line[1] and line[3]=='t':
                foil_fa+=1
            if 'd' in line[1] and line[3]=='f':
                foil_cr+=1
            if line [1] == '':
                junk+=1
            
    target_hit = target_hit
    target_miss = target_miss
    lure_cr = lure_cr
    lure_fa = lure_fa
    target_hit_rate = target_hit / (target)
    fa_cr_rate = foil_cr / (foil)
    target_miss_rate = target_miss/target
    lure_cr_rate = lure_cr/lure
    lure_fa_rate = lure_fa/lure
    foil_cr_rate = foil_cr/foil
    foil_fa_rate = foil_fa/foil
    cr = lure_cr + foil_cr
    cr_rate = (lure_cr + foil_cr)/(lure + foil)
    fa = lure_fa + foil_fa
    fa_rate = (lure_fa + foil_fa)/(lure + foil)
    
    if target_hit_rate == 1: 
        target_hit_rate = (target - 0.5) / (target)
    if target_hit_rate == 0: 
        target_hit_rate = 0.5 / (target)
        
    if foil_fa_rate == 1: 
        foil_fa_rate = (foil - 0.5) / (foil)
    if foil_fa_rate == 0: 
        foil_fa_rate = 0.5 / (foil)    
    
    dprime = Z(target_hit_rate) - Z(foil_fa_rate)
    dprime_lures = Z(target_hit_rate) - Z(lure_fa_rate)
    
    LDI = lure_cr_rate - target_miss_rate
    
    LDI_reviewer = lure_cr_rate - foil_fa_rate
    
    beta = math.exp((Z(fa_rate)**2 - Z(target_hit_rate)**2) / 2)
    c = -(Z(target_hit_rate) + Z(fa_rate)) / 2
    ad = norm.cdf(dprime / math.sqrt(2))
    
    # Append our big data frame with each subject's data
    
    sub_all_group_older_desc_perceptual = sub_all_group_older_desc_perceptual.append(pd.DataFrame\
    ({'subject': subject, 'target_hit': target_hit, 'target_miss': target_miss,\
      'lure_cr': lure_cr, 'lure_fa': lure_fa,\
      'foil_cr': foil_cr, 'foil_fa':foil_fa,\
      'target_hit_rate': target_hit_rate, 'lure_cr_rate': lure_cr_rate,'foil_cr_rate': foil_cr_rate,\
      'lure_fa_rate': lure_fa_rate, 'foil_fa_rate':foil_fa_rate,\
      'cr': cr, 'cr_rate': cr_rate, 'fa': fa,'fa_rate': fa_rate,\
      'dprime':dprime, 'dprime_lures':dprime_lures, 'LDI':LDI, 'LDI_reviewer':LDI_reviewer, 'beta':beta, 'c':c, 'ad':ad}, index=[0]), ignore_index=True)

    #Append performance summary file
    perceptual_summary.write("%d,%d,%d,%d,%d,%d,%d,%d,%d,%d\n\n\n\n" %(target,target_hit,target_miss,lure,lure_cr,lure_fa,foil,foil_cr,foil_fa,junk))
    perceptual_summary.write("targets\n\ntarget_hit_rate,target_miss_rate\n")
    perceptual_summary.write("%.3f,%.3f\n\n\n"%(target_hit_rate,target_miss_rate))
    perceptual_summary.write("lures\n\nlure_cr_rate,lure_fa_rate\n")
    perceptual_summary.write("%.3f,%.3f\n\n\n"%(lure_cr_rate,lure_fa_rate))
    perceptual_summary.write("foils\n\nfoil_cr_rate,foil_fa_rate\n")
    perceptual_summary.write("%.3f,%.3f\n\n\n"%(foil_cr_rate,foil_fa_rate))
    perceptual_summary.close()
    print("perceptual_summary created successfully for %s" %(subject))

perceptual_summary created successfully for 201
perceptual_summary created successfully for 202
perceptual_summary created successfully for 203
perceptual_summary created successfully for 204
perceptual_summary created successfully for 205
perceptual_summary created successfully for 206
perceptual_summary created successfully for 207
perceptual_summary created successfully for 208
perceptual_summary created successfully for 209
perceptual_summary created successfully for 210
perceptual_summary created successfully for 211
perceptual_summary created successfully for 212
perceptual_summary created successfully for 213
perceptual_summary created successfully for 214
perceptual_summary created successfully for 215
perceptual_summary created successfully for 216
perceptual_summary created successfully for 217
perceptual_summary created successfully for 218
perceptual_summary created successfully for 219
perceptual_summary created successfully for 220
perceptual_summary created successfully 

### Younger Adults 

In [9]:
sublist_younger = pd.read_csv(os.path.join(base_dir, 'code/sub-all_group-younger_desc-subject_list.txt'), header=None)
sublist_younger = sublist_younger[0].values.tolist()
sublist_younger

[301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321]

### Narrative Task

In [11]:
Z = norm.ppf

sub_all_group_younger_desc_narrative = pd.DataFrame(columns=['subject', 'target_hit', 'lure_cr','foil_cr'])

for subject in sublist_younger: 
    narrative_summary = open(os.path.join(output_dir, f'sub-{subject}_group-younger_task-recognition_desc-narrative_results.txt'), 'w')
    narrative_summary.write('subject: %d\n' %(subject))
    narrative_summary.write('performance_summary\n\n\n\nraw_data\n\n')
    narrative_summary.write('target,target_hit,target_miss,lure,lure_cr,lure_fa,foil,foil_cr,foil_fa,junk\n')
    
    performance=[]
    target=0
    target_hit=0
    target_miss=0
    lure=0
    lure_cr=0
    lure_fa=0
    foil=0
    foil_cr=0
    foil_fa=0
    junk=0
    
    with open(os.path.join(data_dir, f'sub-{subject}_group-younger_task-recognition_desc-narrative.txt')) as fd:
        for line in islice(csv.reader(fd), 5, None):
            performance.append([line[0],line[1],line[2],line[3],line[4]])
            if 't' in line[1]:
                target+=1
            if 't' in line[1] and line[3]=='t':
                target_hit+=1
            if 't' in line[1] and line[3]=='f':
                target_miss+=1
            # Start move to lure trials
            if 's' in line[1]:
                lure+=1
            if 's' in line[1] and line[3]=='t':
                lure_fa+=1
            if 's' in line[1] and line[3]=='f':
                lure_cr+=1
            if 'n' in line[1]:
                foil+=1
            if 'n' in line[1] and line[3]=='t':
                foil_fa+=1
            if 'n' in line[1] and line[3]=='f':
                foil_cr+=1
            if line [1] == '':
                junk+=1
            
    target_hit = target_hit
    target_miss = target_miss
    lure_cr = lure_cr
    lure_fa = lure_fa
    target_hit_rate = target_hit / (target)
    fa_cr_rate = foil_cr / (foil)
    target_miss_rate = target_miss/target
    lure_cr_rate = lure_cr/lure
    lure_fa_rate = lure_fa/lure
    foil_cr_rate = foil_cr/foil
    foil_fa_rate = foil_fa/foil
    cr = lure_cr + foil_cr
    cr_rate = (lure_cr + foil_cr)/(lure + foil)
    fa = lure_fa + foil_fa
    fa_rate = (lure_fa + foil_fa)/(lure + foil)
    
    if target_hit_rate == 1: 
        target_hit_rate = (target - 0.5) / (target)
    if target_hit_rate == 0: 
        target_hit_rate = 0.5 / (target)
        
    if foil_fa_rate == 1: 
        foil_fa_rate = (foil - 0.5) / (foil)
    if foil_fa_rate == 0: 
        foil_fa_rate = 0.5 / (foil)    
    
    dprime = Z(target_hit_rate) - Z(foil_fa_rate)
    dprime_lures = Z(target_hit_rate) - Z(lure_fa_rate)
    
    LDI = lure_cr_rate - target_miss_rate
    
    LDI_reviewer = lure_cr_rate - foil_fa_rate
    
    beta = math.exp((Z(fa_rate)**2 - Z(target_hit_rate)**2) / 2)
    c = -(Z(target_hit_rate) + Z(fa_rate)) / 2
    ad = norm.cdf(dprime / math.sqrt(2))
    
    # Append our big data frame with each subject's data
    
    sub_all_group_younger_desc_narrative = sub_all_group_younger_desc_narrative.append(pd.DataFrame\
    ({'subject': subject, 'target_hit': target_hit, 'target_miss': target_miss,\
      'lure_cr': lure_cr, 'lure_fa': lure_fa,\
      'foil_cr': foil_cr, 'foil_fa':foil_fa,\
      'target_hit_rate': target_hit_rate, 'lure_cr_rate': lure_cr_rate,'foil_cr_rate': foil_cr_rate,\
      'lure_fa_rate': lure_fa_rate, 'foil_fa_rate':foil_fa_rate,\
      'cr': cr, 'cr_rate': cr_rate, 'fa': fa,'fa_rate': fa_rate,\
      'dprime':dprime, 'dprime_lures':dprime_lures, 'LDI':LDI, 'LDI_reviewer':LDI_reviewer, 'beta':beta, 'c':c, 'ad':ad}, index=[0]), ignore_index=True)

#     Append performance summary file
    narrative_summary.write("%d,%d,%d,%d,%d,%d,%d,%d,%d,%d\n\n\n\n" %(target,target_hit,target_miss,lure,lure_cr,lure_fa,foil,foil_cr,foil_fa,junk))
    narrative_summary.write("targets\n\ntarget_hit_rate,target_miss_rate\n")
    narrative_summary.write("%.3f,%.3f\n\n\n"%(target_hit_rate,target_miss_rate))
    narrative_summary.write("lures\n\nlure_cr_rate,lure_fa_rate\n")
    narrative_summary.write("%.3f,%.3f\n\n\n"%(lure_cr_rate,lure_fa_rate))
    narrative_summary.write("foils\n\nfoil_cr_rate,foil_fa_rate\n")
    narrative_summary.write("%.3f,%.3f\n\n\n"%(foil_cr_rate,foil_fa_rate))
    narrative_summary.close()
    print("narrative_summary created successfully for %s" %(subject))

narrative_summary created successfully for 301
narrative_summary created successfully for 302
narrative_summary created successfully for 303
narrative_summary created successfully for 304
narrative_summary created successfully for 305
narrative_summary created successfully for 306
narrative_summary created successfully for 307
narrative_summary created successfully for 308
narrative_summary created successfully for 309
narrative_summary created successfully for 310
narrative_summary created successfully for 311
narrative_summary created successfully for 312
narrative_summary created successfully for 313
narrative_summary created successfully for 314
narrative_summary created successfully for 315
narrative_summary created successfully for 316
narrative_summary created successfully for 317
narrative_summary created successfully for 318
narrative_summary created successfully for 319
narrative_summary created successfully for 320
narrative_summary created successfully for 321


In [12]:
sub_all_group_younger_desc_perceptual = pd.DataFrame(columns=['subject', 'target_hit', 'lure_cr','foil_cr'])

for subject in sublist_younger: 
    perceptual_summary = open(os.path.join(output_dir, f'sub-{subject}_group-younger_task-recognition_desc-perceptual_results.txt'), 'w')
    perceptual_summary.write('subject: %d\n' %(subject))
    perceptual_summary.write('performance_summary\n\n\n\nraw_data\n\n')
    perceptual_summary.write('target,target_hit,target_miss,lure,lure_cr,lure_fa,foil,foil_cr,foil_fa,junk\n')
    
    performance=[]
    target=0
    target_hit=0
    target_miss=0
    lure=0
    lure_cr=0
    lure_fa=0
    foil=0
    foil_cr=0
    foil_fa=0
    junk=0
    
    with open(os.path.join(data_dir, f'sub-{subject}_group-younger_task-recognition_desc-perceptual.txt')) as fd:
        for line in islice(csv.reader(fd), 5, None):
            performance.append([line[0],line[1],line[2],line[3],line[4]])
            if 't' in line[1]:
                target+=1
            if 't' in line[1] and line[3]=='t':
                target_hit+=1
            if 't' in line[1] and line[3]=='f':
                target_miss+=1
            # Start move to lure trials
            if 's' in line[1]:
                lure+=1
            if 's' in line[1] and line[3]=='t':
                lure_fa+=1
            if 's' in line[1] and line[3]=='f':
                lure_cr+=1
            if 'd' in line[1]:
                foil+=1
            if 'd' in line[1] and line[3]=='t':
                foil_fa+=1
            if 'd' in line[1] and line[3]=='f':
                foil_cr+=1
            if line [1] == '':
                junk+=1
            
    target_hit = target_hit
    target_miss = target_miss
    lure_cr = lure_cr
    lure_fa = lure_fa
    target_hit_rate = target_hit / (target)
    fa_cr_rate = foil_cr / (foil)
    target_miss_rate = target_miss/target
    lure_cr_rate = lure_cr/lure
    lure_fa_rate = lure_fa/lure
    foil_cr_rate = foil_cr/foil
    foil_fa_rate = foil_fa/foil
    cr = lure_cr + foil_cr
    cr_rate = (lure_cr + foil_cr)/(lure + foil)
    fa = lure_fa + foil_fa
    fa_rate = (lure_fa + foil_fa)/(lure + foil)
    
    if target_hit_rate == 1: 
        target_hit_rate = (target - 0.5) / (target)
    if target_hit_rate == 0: 
        target_hit_rate = 0.5 / (target)
        
    if foil_fa_rate == 1: 
        foil_fa_rate = (foil - 0.5) / (foil)
    if foil_fa_rate == 0: 
        foil_fa_rate = 0.5 / (foil)    
    
    dprime = Z(target_hit_rate) - Z(foil_fa_rate)
    dprime_lures = Z(target_hit_rate) - Z(lure_fa_rate)
    
    LDI = lure_cr_rate - target_miss_rate
    
    LDI_reviewer = lure_cr_rate - foil_fa_rate
    
    beta = math.exp((Z(fa_rate)**2 - Z(target_hit_rate)**2) / 2)
    c = -(Z(target_hit_rate) + Z(fa_rate)) / 2
    ad = norm.cdf(dprime / math.sqrt(2))
    
    # Append our big data frame with each subject's data
    
    sub_all_group_younger_desc_perceptual = sub_all_group_younger_desc_perceptual.append(pd.DataFrame\
    ({'subject': subject, 'target_hit': target_hit, 'target_miss': target_miss,\
      'lure_cr': lure_cr, 'lure_fa': lure_fa,\
      'foil_cr': foil_cr, 'foil_fa':foil_fa,\
      'target_hit_rate': target_hit_rate, 'lure_cr_rate': lure_cr_rate,'foil_cr_rate': foil_cr_rate,\
      'lure_fa_rate': lure_fa_rate, 'foil_fa_rate':foil_fa_rate,\
      'cr': cr, 'cr_rate': cr_rate, 'fa': fa,'fa_rate': fa_rate,\
      'dprime':dprime, 'dprime_lures':dprime_lures, 'LDI':LDI, 'LDI_reviewer':LDI_reviewer, 'beta':beta, 'c':c, 'ad':ad}, index=[0]), ignore_index=True)

    #Append performance summary file
    perceptual_summary.write("%d,%d,%d,%d,%d,%d,%d,%d,%d,%d\n\n\n\n" %(target,target_hit,target_miss,lure,lure_cr,lure_fa,foil,foil_cr,foil_fa,junk))
    perceptual_summary.write("targets\n\ntarget_hit_rate,target_miss_rate\n")
    perceptual_summary.write("%.3f,%.3f\n\n\n"%(target_hit_rate,target_miss_rate))
    perceptual_summary.write("lures\n\nlure_cr_rate,lure_fa_rate\n")
    perceptual_summary.write("%.3f,%.3f\n\n\n"%(lure_cr_rate,lure_fa_rate))
    perceptual_summary.write("foils\n\nfoil_cr_rate,foil_fa_rate\n")
    perceptual_summary.write("%.3f,%.3f\n\n\n"%(foil_cr_rate,foil_fa_rate))
    perceptual_summary.close()
    print("perceptual_summary created successfully for %s" %(subject))

perceptual_summary created successfully for 301
perceptual_summary created successfully for 302
perceptual_summary created successfully for 303
perceptual_summary created successfully for 304
perceptual_summary created successfully for 305
perceptual_summary created successfully for 306
perceptual_summary created successfully for 307
perceptual_summary created successfully for 308
perceptual_summary created successfully for 309
perceptual_summary created successfully for 310
perceptual_summary created successfully for 311
perceptual_summary created successfully for 312
perceptual_summary created successfully for 313
perceptual_summary created successfully for 314
perceptual_summary created successfully for 315
perceptual_summary created successfully for 316
perceptual_summary created successfully for 317
perceptual_summary created successfully for 318
perceptual_summary created successfully for 319
perceptual_summary created successfully for 320
perceptual_summary created successfully 

## Append Older and Younger Adults' Output files together

In [13]:
sub_all_group_older_desc_narrative["group"] = "older"
sub_all_group_older_desc_perceptual["group"] = "older"
sub_all_group_younger_desc_narrative["group"] = "younger"
sub_all_group_younger_desc_perceptual["group"] = "younger"

sub_all_group_older_desc_narrative["type"] = "narrative"
sub_all_group_older_desc_perceptual["type"] = "perceptual"
sub_all_group_younger_desc_narrative["type"] = "narrative"
sub_all_group_younger_desc_perceptual["type"] = "perceptual"

sub_all_group_all_desc_narrative = pd.concat([sub_all_group_older_desc_narrative, sub_all_group_younger_desc_narrative])
sub_all_group_all_desc_perceptual = pd.concat([sub_all_group_older_desc_perceptual, sub_all_group_younger_desc_perceptual])

sub_all_group_older = pd.concat([sub_all_group_older_desc_narrative, sub_all_group_older_desc_perceptual])
sub_all_group_younger = pd.concat([sub_all_group_younger_desc_narrative, sub_all_group_younger_desc_perceptual])

sub_all_group_all_desc_all = pd.concat([sub_all_group_all_desc_narrative, sub_all_group_all_desc_perceptual])
sub_all_group_all_desc_all

,subject,target_hit,lure_cr,foil_cr,target_miss,lure_fa,foil_fa,target_hit_rate,lure_cr_rate,foil_cr_rate,...,fa_rate,dprime,dprime_lures,LDI,LDI_reviewer,beta,c,ad,group,type
0,201,27,26,30,3.0,4.0,0.0,0.900000,0.866667,1.000000,...,0.066667,3.409597,2.392323,0.766667,0.850000,1.357225,0.109767,0.992044,older,narrative
1,202,29,20,30,1.0,10.0,0.0,0.966667,0.666667,1.000000,...,0.166667,3.961960,2.264642,0.633333,0.650000,0.297105,-0.433247,0.997457,older,narrative
2,203,24,23,29,6.0,7.0,1.0,0.800000,0.766667,0.966667,...,0.133333,2.675536,1.569535,0.566667,0.733333,1.300493,0.134575,0.970747,older,narrative
3,204,21,26,30,9.0,4.0,0.0,0.700000,0.866667,1.000000,...,0.066667,2.652446,1.635172,0.566667,0.850000,2.688898,0.488343,0.969642,older,narrative
4,205,25,23,29,5.0,7.0,1.0,0.833333,0.766667,0.966667,...,0.133333,2.801336,1.695335,0.600000,0.733333,1.160620,0.071675,0.976196,older,narrative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,317,29,18,30,1.0,12.0,0.0,0.966667,0.600000,1.000000,...,0.200000,3.961960,2.087262,0.566667,0.583333,0.265150,-0.496147,0.997457,younger,perceptual
17,318,28,24,30,2.0,6.0,0.0,0.933333,0.800000,1.000000,...,0.100000,3.629131,2.342707,0.733333,0.783333,0.736797,-0.109767,0.994859,younger,perceptual
18,319,30,28,30,0.0,2.0,0.0,0.983333,0.933333,1.000000,...,0.033333,4.256090,3.629131,0.933333,0.916667,0.558407,-0.147065,0.998692,younger,perceptual
19,320,28,28,30,2.0,2.0,0.0,0.933333,0.933333,1.000000,...,0.033333,3.629131,3.002172,0.866667,0.916667,1.741927,0.166414,0.994859,younger,perceptual


## Add neuropsychological test results on exisiting dataframe

In [14]:
neuropsych = pd.read_csv(os.path.join(base_dir, 'data/neuropsych/neuropsych_scores.csv'))
#neuropsych

In [15]:
sub_all_group_older = pd.merge(sub_all_group_older, neuropsych, left_on='subject', right_on='subject')

In [16]:
sub_all_group_all_desc_recog_neuropsych = pd.concat([sub_all_group_older, sub_all_group_younger])
#sub_all_group_all_desc_recog_neuropsych

In [17]:
sub_all_group_all_desc_recog_neuropsych["group_type"] = sub_all_group_all_desc_recog_neuropsych["group"] + "_" +sub_all_group_all_desc_recog_neuropsych["type"]
#sub_all_group_all_desc_recog_neuropsych

## Export appended dataframe to CSV file

In [18]:
sub_all_group_younger_desc_narrative.to_csv(os.path.join(output_dir, f'sub-all_group-younger_task-recognition_desc-narrative_results.csv'), index = None, header=True)
sub_all_group_younger_desc_perceptual.to_csv(os.path.join(output_dir, f'sub-all_group-younger_task-recognition_desc-perceptual_results.csv'), index = None, header=True)
sub_all_group_older_desc_narrative.to_csv(os.path.join(output_dir, f'sub-all_group-older_task-recognition_desc-narrative_results.csv'), index = None, header=True)
sub_all_group_older_desc_perceptual.to_csv(os.path.join(output_dir, f'sub-all_group-older_task-recognition_desc-perceptual_results.csv'), index = None, header=True)
sub_all_group_all_desc_narrative.to_csv(os.path.join(output_dir, f'sub-all_group-all_task-recognition_desc-narrative_results.csv'), index = None, header=True)
sub_all_group_all_desc_perceptual.to_csv(os.path.join(output_dir, f'sub-all_group-all_task-recognition_desc-perceptual_results.csv'), index = None, header=True)
sub_all_group_all_desc_all.to_csv(os.path.join(output_dir, f'sub-all_group-all_task-recognition_desc-all_results.csv'), index = None, header=True)
sub_all_group_all_desc_recog_neuropsych.to_csv(os.path.join(output_dir, f'sub-all_group-all_task-recognition_desc-all_results_neuropsych.csv'), index = None, header=True)